In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF,IDF,Tokenizer
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("logistics regression").getOrCreate()

In [2]:
# 创建一个简单的DataFrame，每一个句子代表一个文档。
sentenceData = spark.createDataFrame([(0, "I heard about Spark and I love Spark"),(0, "I wish Java could use case classes"),(1, "Logistic regression models are neat")]).toDF("label", "sentence")

In [3]:
# 使用分解器Tokenizer把句子划分为单个词语。
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

In [4]:
# 对每一个句子（词袋），我们使用HashingTF将句子转换为特征向量,这里设置哈希表的桶数为2000。
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# 分词序列被变换成一个稀疏特征向量，其中每个单词都被散列成了一个不同的索引值，特征向量在某一维度上的值即该词汇在文档中出现的次数。

In [5]:
# 使用IDF来对单纯的词频特征向量进行修正，使其更能体现不同词汇对文本的区别能力，IDF是一个Estimator，调用fit()方法并将词频向量传入，即产生一个IDFModel。。这种转换通常可以提高使用文本特征的性能。
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

In [6]:
# IDFModel是一个Transformer，调用它的transform()方法，即可得到每一个单词对应的TF-IDF度量值
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(20,[0,5,9,13,17]...|
|    0|(20,[2,7,9,13,15]...|
|    1|(20,[4,6,13,15,18...|
+-----+--------------------+

